In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import nltk
%matplotlib inline

from bs4 import BeautifulSoup
import requests
import re
import pyinputplus as pyip
import requests
import datetime as dt
from datetime import datetime
import pickle
import random
import math
import string

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 150)

In [2]:
# Constants

YDS_GRADES_FULL = ['5.0', '5.1', '5.2', '5.3', '5.4', '5.5', '5.6', '5.7', '5.7+', '5.8-', '5.8', '5.8+', '5.9-', '5.9', '5.9+', '5.10a', '5.10-', '5.10a/b', '5.10b', '5.10', '5.10b/c', '5.10c', '5.10+', '5.10c/d', '5.10d', '5.11a', '5.11-', '5.11a/b', '5.11b', '5.11', 
                   '5.11b/c', '5.11c', '5.11+', '5.11c/d', '5.11d', '5.12a', '5.12-', '5.12a/b', '5.12b', '5.12', '5.12b/c', '5.12c', '5.12+', '5.12c/d', '5.12d', '5.13a', '5.13-', '5.13a/b', '5.13b', '5.13', '5.13b/c', '5.13c', '5.13+', '5.13c/d', '5.13d', '5.14a', 
                   '5.14-', '5.14a/b', '5.14b', '5.14', '5.14b/c', '5.14c', '5.14+', '5.14c/d', '5.14d', '5.15a', '5.15-', '5.15a/b', '5.15b', '5.15', '5.15b/c' '5.15c', '5.15+', '5.15c/d', '5.15d'] 
YDS_GRADES_LETTER = ['5.0', '5.1', '5.2', '5.3', '5.4', '5.5', '5.6', '5.7', '5.8', '5.9', '5.10a', '5.10b', '5.10c', '5.10d', '5.11a', '5.11b', '5.11c', '5.11d', '5.12a', '5.12b', '5.12c', '5.12d', '5.13a', '5.13b', '5.13c', '5.13d', '5.14a', '5.14b', '5.14c', '5.14d', '5.15a', '5.15b', '5.15c', '5.15d']
YDS_GRADES_SIGN = ['5.0', '5.1', '5.2', '5.3', '5.4', '5.5', '5.6', '5.7', '5.7+', '5.8-', '5.8', '5.8+', '5.9-', '5.9', '5.9+', '5.10-', '5.10', '5.10+', '5.11-', '5.11', '5.11+', '5.12-', '5.12', '5.12+', '5.13-', '5.13', '5.13+', '5.14-', '5.14', '5.14+', '5.15-', '5.15', '5.15+']

V_GRADES_FULL = ['v-easy', 'v0-', 'v0', 'v0+', 'v0-1', 'v1-', 'v1', 'v1+', 'v1-2', 'v2-', 'v2', 'v2+', 'v2-3', 'v3-', 'v3', 'v3+', 'v3-4', 'v4-', 'v4', 'v4+', 'v4-5', 'v5-', 'v5', 'v5+', 'v5-6', 'v6-', 'v6', 'v6+', 'v6-7', 'v7-', 'v7', 'v7+', 'v7-8', 'v8-', 'v8', 'v8+', 'v8-9', 
                 'v9-', 'v9', 'v9+', 'v9-10', 'v10-', 'v10', 'v10+', 'v10-11', 'v11-', 'v11', 'v11+', 'v11-12', 'v12-', 'v12', 'v12+', 'v12-13', 'v13-', 'v13', 'v13+', 'v13-14', 'v14-', 'v14', 'v14+', 'v14-15', 'v15-', 'v15', 'v15+', 'v15-16', 'v16-', 'v16', 'v16+', 'v16-17', 'v17-', 'v17']
V_GRADES_SIGN = ['v-easy', 'v0-', 'v0', 'v0+', 'v1-', 'v1', 'v1+', 'v2-', 'v2', 'v2+', 'v3-', 'v3', 'v3+', 'v4-', 'v4', 'v4+', 'v5-', 'v5', 'v5+', 'v6-', 'v6', 'v6+', 'v7-', 'v7', 'v7+', 'v8-', 'v8', 'v8+', 
                 'v9-', 'v9', 'v9+', 'v10-', 'v10', 'v10+', 'v11-', 'v11', 'v11+', 'v12-', 'v12', 'v12+', 'v13-', 'v13', 'v13+', 'v14-', 'v14', 'v14+', 'v15-', 'v15', 'v15+', 'v16-', 'v16', 'v16+', 'v17-', 'v17']
V_GRADES_BASIC = ['v-easy', 'v0', 'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v12', 'v13', 'v14', 'v15', 'v16', 'v17']

RISK_GRADES = ['pg', 'pg13', 'r', 'x']

CLEAN_SEND = ['onsight', 'flash', 'redpoint', 'pinkpoint']
CLEAN_SEND_FIRST = ['onsight', 'flash']
CLEAN_SEND_WORKED = ['redpoint', 'pinkpoint']
TICK_OPTIONS = ['solo', 'tr', 'follow', 'lead', 'fell/hung', 'onsight', 'flash', 'redpoint', 'pinkpoint']

In [20]:
# Allow user to select data source

datasource_choices = ["csv", "link"]
resp = pyip.inputChoice(datasource_choices)
if resp == datasource_choices[0]:
    while True:
        try:
            resp = pyip.inputFilepath(prompt='input csv file path\n', mustExist=True)
            df = pd.read_csv(resp)
        except Exception as err:
            print(err)
            continue
        break
if resp == datasource_choices[1]:
    while True:
        try:
            resp = input('input link to profile of format "https://www.mountainproject.com/user/..."\n')
            df = pd.read_csv(f'{resp}/tick-export')
        except Exception as err:
            print(err)
            continue
        break

Please select one of: csv, link


In [18]:
df = pd.read_csv('./ticks.csv')
df.head()

,Date,Route,Rating,Notes,URL,Pitches,Location,Avg Stars,Your Stars,Style,Lead Style,Route Type,Your Rating,Length,Rating Code
0,2022-11-05,Urban Struggle,5.12a,Got it clean! Felt solid and really dialed thr...,https://www.mountainproject.com/route/10599853...,1,California > Los Angeles Basin > Santa Monica ...,3.4,-1,Lead,Redpoint,Sport,NaN,50.0,6600
1,2022-11-05,Urban Struggle,5.12a,First effort greased out of the crux pocket.,https://www.mountainproject.com/route/10599853...,1,California > Los Angeles Basin > Santa Monica ...,3.4,-1,Lead,Fell/Hung,Sport,NaN,50.0,6600
2,2022-11-05,Directpissima,5.11a,The steep stuff up high is so sick. I realized...,https://www.mountainproject.com/route/10592801...,1,California > Los Angeles Basin > Santa Monica ...,2.4,-1,Lead,Redpoint,Sport,NaN,50.0,4600
3,2022-11-05,Unnamed,5.10d,"Very soft for the grade, I would say .10b and ...",https://www.mountainproject.com/route/11939791...,1,California > Los Angeles Basin > Santa Monica ...,2.4,-1,Lead,Onsight,Sport,NaN,45.0,3500
4,2022-11-05,Spiker Monkey,5.10c,Pretty sure this is the line we did. Good fun!,https://www.mountainproject.com/route/11818569...,1,California > Los Angeles Basin > Santa Monica ...,2.5,-1,TR,NaN,TR,NaN,45.0,3200


In [ ]:
# Transform all strings to lower case so we don't have to worry about it for operations. Can be turned to camel case later for presentation.
df = df.applymap(lambda s: s.lower() if type(s) == str else s)

# Apostrophe's are html coded in the notes section for some reason.
df['Notes'] = df['Notes'].apply(lambda x: str(x).replace('&#39;',"'"))

# 'Date' to datetype
df['Date'] = pd.to_datetime(df['Date'])

# Remove all aid, ice, snow, TR only,and trad/boulder climbing route types as they are not relevant.
df = df[df['Route Type'].str.contains(r'aid|ice|snow') != True]
df = df[df['Route Type'].str.fullmatch(r'tr') != True] #if this is just a partial match it will detech "trad" too!
df = df[df['Route Type'].str.contains(r'trad') & df['Route Type'].str.contains(r'boulder') != True]

# "trad, sport" goes to "trad". If it uses gear it's trad!
df.loc[df['Route Type'].str.contains(r'trad') & df['Route Type'].str.contains(r'sport'), 'Route Type'] = 'trad'

# "x, alpine" and "x, tr" goes to "x" Alpine and tr tags are not useful.
def rem_route_el_from_list(ousted, seperator):
    el_rem_subset = df['Route Type'].str.contains(ousted) == True
    df.loc[el_rem_subset, 'Route Type'] = df[el_rem_subset]['Route Type'].apply(lambda row: [val for val in row.split(seperator) if val != ousted]).apply(lambda x: ", ".join(x))

rem_route_el_from_list('alpine', ', ')
rem_route_el_from_list('tr', ', ')

# Extract route unique identifier from URL and create a new column for it.
df.insert(len(df.columns),'Route ID','')
df['Route ID'] = df['URL'].apply(lambda x: x.split('/')[4])

# Create original rating and length archive to compare against or undo changes.
df.insert(df.columns.get_loc('Rating'),'Original Rating',df['Rating'])

# Change YDS-Vgrade combos to just Vgrade. They are most likely boulders, so a bouldering grade is relevant.
subset = df['Rating'].apply(lambda row: [val for val in row.split() if val in V_GRADES_FULL]).astype(bool)  & df['Rating'].apply(lambda row: [val for val in row.split() if val in YDS_GRADES_FULL]).astype(bool) == True
df.loc[subset, 'Rating'] = df[subset]['Rating'].apply(lambda x: x.split()[1])

# Seperate risk rating to new column
df.insert(df.columns.get_loc('Rating')+1,'Risk','')
df['Risk'] = df['Rating'].apply(lambda row: [val for val in row.split() if val in RISK_GRADES]).apply(lambda x: "".join(x))
# Reduce Rating column to just rating
df['Rating'] = df['Rating'].apply(lambda row: [val for val in row.split()][0])
# Displays all rows where risk ratings have been seperated
# df[df['Risk'].astype(bool)]

df.insert(len(df.columns),'Flash/Onsight','')
df.insert(len(df.columns),'Worked Clean','')
df.insert(len(df.columns),'Grade Breakthrough','')
df.insert(len(df.columns),'Attempts','')

In [ ]:
# Download page of each climb

df.insert(df.columns.get_loc('Pitches')+1,'Listed Pitches','')
df.insert(len(df.columns),'re mainpage','')
df.insert(len(df.columns),'re statpage','')
df.insert(len(df.columns),'Statpage df','')

def insert_str_to_address(url, insert_phrase):
    str_list = url.split('/')
    str_list.insert(4, insert_phrase)
    return '/'.join(str_list)

def page_download(url):
    try:
        res = requests.get(url, timeout=10)
    except Exception as e:
        print(e)
    print(res)
    return res

df['re mainpage'] = df['URL'].apply(page_download)
df['re statpage'] = df['URL'].apply(lambda x: page_download(insert_str_to_address(x, 'stats')))

#TODO consider applying this function only to unique row IDs to speed it up, then simply copy and paste it among repeats.

In [209]:
# Let's analyze the mainpage for listed default pitch lengths. This is not included in the tick export but is helpful in determining user tick methodology

def get_pitches(res):
    soup = BeautifulSoup(res.text, 'html.parser')
    route_type_text = str(soup.find(class_="description-details").find_all('td')[1])
    pitch_search = re.search(r'\d+ pitches',route_type_text)
    if str(type(pitch_search)) == "<class 'NoneType'>":
        num_pitches = 1
    else:
        num_pitches = pitch_search.group(0).split(' ')[0]
    return int(num_pitches)

df['Listed Pitches'] = df['re mainpage'].apply(get_pitches)


In [361]:
# Let's save the now scraped dataframe to a pickle file
df.to_pickle('../Data_Archive/df_archive_base')

In [3]:
# To load the new pickle file
picklefile = open('../Data_Archive/df_archive_base', 'rb')
df = pickle.load(picklefile)

In [4]:
# The first choice is what type of climbing to analyze.

choices = ["roped", "sport", "trad", 'boulder']
route_type_selection = pyip.inputChoice(choices)
if route_type_selection == choices[0]:
    df = df[(df['Route Type'] == 'sport') | (df['Route Type'] == 'trad')]
if route_type_selection == choices[1]:
    df = df[df['Route Type'] == 'sport']
if route_type_selection == choices[2]:
    df = df[df['Route Type'] == 'trad']
if route_type_selection == choices[3]:
    df = df[df['Route Type'] == 'boulder']

# Then ask if the user would like to ignore climbs below a certain grade
min_grade_included = pyip.inputChoice(prompt='Select a minimum grade to include (blank includes all):\n', choices=YDS_GRADES_LETTER, blank=True)
if min_grade_included != '':
    if route_type_selection == 'roped' or 'sport' or 'trad':
        df = df[df['Rating'].isin(YDS_GRADES_FULL[YDS_GRADES_FULL.index(min_grade_included):])] # the : here is key, it makes it so that it includes all grades above the min included.
    if route_type_selection == 'boulder':
        df = df[df['Rating'].isin(V_GRADES_FULL[V_GRADES_FULL.index(min_grade_included):])]

Please select one of: roped, sport, trad, boulder
Select a minimum grade to include (blank includes all):


In [ ]:
# Handle route length outliers
if route_type_selection == 'boulder':
    MIN_LENGTH = 0
    MAX_LENGTH = 55 #"Too Tall to Fall" is 50'

if route_type_selection == 'roped' or 'sport' or 'trad':
    MIN_LENGTH = 25
    MAX_LENGTH = 4500 #Trango towers are 4,300' tall

length_outliers = df[(df['Length'] <= MIN_LENGTH) | (df['Length'] >= MAX_LENGTH)]
for loop_count, (index, data) in enumerate(length_outliers.iterrows()):
    updated_length = pyip.inputNum(f"[{loop_count+1}/{length_outliers.shape[0]}] Input Correct Length for Route: {data['Route']} (Currently {data['Length']}ft):\n", min=MIN_LENGTH, max=MAX_LENGTH)
    df.at[index, 'Length'] = updated_length

# Fill empty route lengths
length_missing = df[df['Length'].isnull()]
for loop_count, (index, data) in enumerate(length_missing.iterrows()):
    updated_length = pyip.inputNum(f"[{loop_count+1}/{length_missing.shape[0]}] Input Estimated Length for Route: {data['Route']}:\n", min=MIN_LENGTH, max=MAX_LENGTH)
    df.at[index, 'Length'] = updated_length

#TODO allow user to select "assign median height from area"

In [ ]:
if route_type_selection == "roped" or "sport" or "trad":
    #Ask user if they would prefer to round down, round up, round even by random, or hand-determine
    grademoderatemap = {'5.6-': '5.6', '5.6+': '5.6', '5.7-': '5.7', '5.7+': '5.7', '5.8-': '5.8', '5.8+': '5.8', '5.9-': '5.9', '5.9+': '5.9'}
    gradedownmap = {'5.10a/b': '5.10a', '5.10-': '5.10a', '5.10b/c': '5.10b', '5.10': '5.10b', '5.10c/d': '5.10c', '5.10+': '5.10c',
                '5.11a/b': '5.11a', '5.11-': '5.11a', '5.11b/c': '5.11b', '5.11': '5.11b', '5.11c/d': '5.11c', '5.11+': '5.11c',
                '5.12a/b': '5.12a', '5.12-': '5.12a', '5.12b/c': '5.12b', '5.12': '5.12b', '5.12c/d': '5.12c', '5.12+': '5.12c',
                '5.13a/b': '5.13a', '5.13-': '5.13a', '5.13b/c': '5.13b', '5.13': '5.13b', '5.13c/d': '5.13c', '5.13+': '5.13c',
                '5.14a/b': '5.14a', '5.14-': '5.14a', '5.14b/c': '5.14b', '5.14': '5.14b', '5.14c/d': '5.14c', '5.14+': '5.14c',
                '5.15a/b': '5.15a', '5.15-': '5.15a', '5.15b/c': '5.15b', '5.15': '5.15b', '5.15c/d': '5.15c', '5.15+': '5.15c',
    }
    gradeupmap = {'5.10a/b': '5.10b', '5.10-': '5.10b', '5.10b/c': '5.10c', '5.10': '5.10c', '5.10c/d': '5.10d', '5.10+': '5.10d',
                '5.11a/b': '5.11b', '5.11-': '5.11b', '5.11b/c': '5.11c', '5.11': '5.11c', '5.11c/d': '5.11d', '5.11+': '5.11d',
                '5.12a/b': '5.12b', '5.12-': '5.12b', '5.12b/c': '5.12c', '5.12': '5.12c', '5.12c/d': '5.12d', '5.12+': '5.12d',
                '5.13a/b': '5.13b', '5.13-': '5.13b', '5.13b/c': '5.13c', '5.13': '5.13c', '5.13c/d': '5.13d', '5.13+': '5.13d',
                '5.14a/b': '5.14b', '5.14-': '5.14b', '5.14b/c': '5.14c', '5.14': '5.14c', '5.14c/d': '5.14d', '5.14+': '5.14d',
                '5.15a/b': '5.15b', '5.15-': '5.15b', '5.15b/c': '5.15c', '5.15': '5.15c', '5.15c/d': '5.15d', '5.15+': '5.15d',
    }

    grade_homo_choice = ['round down', 'round up', 'round evenly by random', 'hand determine']
    resp = pyip.inputChoice(grade_homo_choice)

    def grademoderate():
        grade_change_subset = df['Rating'].isin(list(grademoderatemap.keys()))
        df.loc[grade_change_subset, 'Rating'] = df[grade_change_subset]['Rating'].map(grademoderatemap)

    def grade_split(upmap, downmap):
        grade_change_subset = df['Rating'].isin(list(upmap.keys()))
        grade_change_subset_df = df[grade_change_subset]
        for grade in grade_change_subset_df['Rating'].unique():
            to_change = grade_change_subset_df[grade_change_subset_df['Rating'] == grade]
            changed_up = to_change.sample(frac=0.5)['Rating'].map(upmap)
            df.loc[changed_up.index, 'Rating'] = changed_up
        grade_change_subset = df['Rating'].isin(list(downmap.keys()))
        grade_change_subset_df = df[grade_change_subset]
        for grade in grade_change_subset_df['Rating'].unique():
            to_change = grade_change_subset_df[grade_change_subset_df['Rating'] == grade]
            changed_down = to_change['Rating'].map(downmap)
            df.loc[changed_down.index, 'Rating'] = changed_down

    if resp == grade_homo_choice[0]:
        grademoderate()
        grade_change_subset = df['Rating'].isin(list(gradedownmap.keys()))
        df.loc[grade_change_subset, 'Rating'] = df[grade_change_subset]['Rating'].map(gradedownmap)
    if resp == grade_homo_choice[1]:
        grademoderate()
        grade_change_subset = df['Rating'].isin(list(gradeupmap.keys()))
        df.loc[grade_change_subset, 'Rating'] = df[grade_change_subset]['Rating'].map(gradeupmap)
    if resp == grade_homo_choice[2]:
        grademoderate()
        grade_split(gradeupmap,gradedownmap)
    if resp == grade_homo_choice[3]:
        needs_grade_corr = df[df['Rating'].isin(list(grademoderatemap.keys()) + list(gradedownmap.keys()))]
        for loop_count, (index, data) in enumerate(needs_grade_corr.iterrows()):
            updated_grade = pyip.inputChoice(prompt=f"[{loop_count+1}/{needs_grade_corr.shape[0]}] Input Grade Correction For: {data['Route'].title()}:\n", choices=YDS_GRADES_LETTER)
            df.at[index, 'Rating'] = updated_grade

if route_type_selection == "Bouldering":
    gradedownmap = {'v0-1': 'v0', 'v1-2': 'v1', 'v2-3': 'v2', 'v3-4': 'v3', 'v4-5': 'v4', 'v5-6': 'v5', 'v6-7': 'v6', 'v7-8': 'v7', 'v8-9': 'v8', 'v9-10': 'v9', 'v10-11': 'v10', 'v11-12': 'v11', 'v12-13': 'v12', 'v13-14': 'v13', 'v14-15': 'v14', 'v15-16':'v15', 'v16-17': 'v16'}
    gradeupmap = {'v0-1': 'v1', 'v1-2': 'v2', 'v2-3': 'v3', 'v3-4': 'v4', 'v4-5': 'v5', 'v5-6': 'v6', 'v6-7': 'v7', 'v7-8': 'v8', 'v8-9': 'v9', 'v9-10': 'v10', 'v10-11': 'v11', 'v11-12': 'v12', 'v12-13': 'v13', 'v13-14': 'v14', 'v14-15': 'v15', 'v15-16':'v16', 'v16-17': 'v17'}
    gradeconmap = {'v0-': 'v0', 'v0+': 'v0', 'v1-': 'v1', 'v1+': 'v1', 'v2-': 'v2', 'v2+': 'v2', 'v3-': 'v3', 'v3+': 'v3', 'v4-': 'v4', 'v4+': 'v4', 'v5-': 'v5', 'v5+': 'v5', 'v6-': 'v6', 'v6+': 'v6', 'v7-': 'v7', 'v7+': 'v7', 'v8-': 'v8', 'v8+': 'v8', 
                    'v9-': 'v9', 'v9+': 'v9', 'v10-': 'v10', 'v10+': 'v10', 'v11-': 'v11', 'v11+': 'v11', 'v12-': 'v12', 'v12+': 'v12', 'v13-': 'v13', 'v13+': 'v13', 'v14-': 'v14', 'v14+': 'v14', 'v15-': 'v15', 'v15+': 'v15', 'v16-': 'v16', 'v16+': 'v16', 'v17-': 'v17', 'v17+': 'v17'}
    
    # Remove all + and - grades
    grade_change_subset = df['Rating'].isin(list(gradeconmap.keys()))
    df.loc[grade_change_subset, 'Rating'] = df[grade_change_subset]['Rating'].map(gradeconmap)
    
    grade_homo_choice = ['round down', 'round up', 'round evenly by random', 'hand determine']
    resp = pyip.inputChoice(grade_homo_choice)
    
    if resp == grade_homo_choice[0]:
        grade_change_subset = df['Rating'].isin(list(gradedownmap.keys()))
        df.loc[grade_change_subset, 'Rating'] = df[grade_change_subset]['Rating'].map(gradedownmap)
    if resp == grade_homo_choice[1]:
        grade_change_subset = df['Rating'].isin(list(gradeupmap.keys()))
        df.loc[grade_change_subset, 'Rating'] = df[grade_change_subset]['Rating'].map(gradeupmap)
    if resp == grade_homo_choice[2]:
        grade_split(gradeupmap,gradedownmap)
    if resp == grade_homo_choice[3]:
        needs_grade_corr = df[df['Rating'].isin(list(grademoderatemap.keys()) + list(gradedownmap.keys()))]
        for loop_count, (index, data) in enumerate(needs_grade_corr.iterrows()):
            updated_grade = pyip.inputChoice(prompt=f"[{loop_count+1}/{needs_grade_corr.shape[0]}] Input Grade Correction For: {data['Route'].title()}:\n", choices=V_GRADES_BASIC)
            df.at[index, 'Rating'] = updated_grade

In [6]:
df.to_pickle('../Data_Archive/df_archive')

In [155]:
# To load the new pickle file
picklefile = open('../Data_Archive/df_archive', 'rb')
df = pickle.load(picklefile)

In [ ]:
# multiple pitch climbs ticked as single pitches are either part of a linkup, first pitch only, or combined into a single pitch. If they are combined into a single pitch we want to ask for grade adjustment
subset = df[(df['Pitches'] == 1) & (df['Listed Pitches'] > 1)]
for loop_count, (index, data) in enumerate(subset.iterrows()):
            updated_grade = pyip.inputChoice(prompt=f"[{loop_count+1}/{subset.shape[0]}] Input Grade Correction For: {data['Route'].title()}:\n", choices=YDS_GRADES_LETTER)
            df.at[index, 'Rating'] = updated_grade

In [32]:
# redpoint ticks with 1x pitch attempt and no prior fell/hung ticks need further data on previous attempts
subset = df[(df['Lead Style'].isin(CLEAN_SEND_WORKED)) & ((df['Pitches'] == 1) | ~df.duplicated('Route'))]
for loop_count, (index, data) in enumerate(subset.iterrows()):
            updated_attempts = pyip.inputnum(prompt=f"[{loop_count+1}/{subset.shape[0]}] : Redpoint without prior attempt data, please input number of failed attempts on {data['Route'].title()}:\n", min=1)
            df.at[index, 'Pitches'] = updated_attempts + 1

In [ ]:
# The "right" way to tick multiple attempts on a single pitch route is to tick a fell/hung for each attempt. Some people may instead tick a fell/hung with multiple pitches for multiple attempts. Someone who sends something first day but in multiple tries
# might also tick a redpoint with multiple pitches to include their attempts leading to that redpoint.

# single pitch climbs ticked as multiple pitches
# 	TR or follow -> break into multiple same day TR
# 	Lead
# 		fell/hung -> break into multiple same day fell/hung
# 		redpoint or pinkpoint -> N-1 fell hung and transform current into 1 pitch
# 		flash or onsight -> likely a mistake, change to 1 pitch

#TODO these just find the subset so far, still need to perform the action listed above.
df[(df['Listed Pitches'] == 1) & (df['Pitches'] > df['Listed Pitches']) & (df['Style'].isin(['tr', 'follow']))]
df[(df['Listed Pitches'] == 1) & (df['Pitches'] > df['Listed Pitches']) & (df['Style'] == 'lead') & (df['Lead Style'] == 'fell/hung')]
df[(df['Listed Pitches'] == 1) & (df['Pitches'] > df['Listed Pitches']) & (df['Style'] == 'lead') & (df['Lead Style'].isin(CLEAN_SEND_WORKED))]
df[(df['Listed Pitches'] == 1) & (df['Pitches'] > df['Listed Pitches']) & (df['Style'] == 'lead') & (df['Lead Style'].isin(CLEAN_SEND_FIRST))]

In [ ]:
# We want to tag important climbs, namely flash/onsights, worked clean routes and grade breakthroughs.

# Tag climbs that were flash/onsight
df.loc[df['Lead Style'].isin(CLEAN_SEND_FIRST), 'Flash/Onsight'] = True

# Create column that flags climbs that were worked. There are three possibilities to consider. We want 1 and 2.
# 1. Worked to clean send, no further sends.
# 2. Worked to clean send, additional attempts.
# 3. Sent clean first try, additional attempts.
df_no_dupe = df[df.duplicated(subset="Route ID", keep=False)] # First we filter for all duplicate entries.
df_all_worked = df_no_dupe.groupby('Route ID').filter(lambda x: ~x['Lead Style'].isin(CLEAN_SEND_FIRST).any()) # Then we remove all groups which have a lead style of flash or onsight to eliminate group 3.
df_worked_sub1 = df_all_worked[df_all_worked.groupby('Route ID')['Lead Style'].apply(lambda x: x.isin(CLEAN_SEND_WORKED))] # fell/hungs and TRs remain, so we take ticks from CLEAN_SEND_WORKED.
df_worked_sub2 = df_worked_sub1.loc[df_worked_sub1.groupby('Route ID')['Date'].idxmin()] # Use only the earliest redpoint to correctly identify the first redpoint.
df.loc[df_worked_sub2.index.values, "Worked Clean"] = True

# Flag grade breakthrough ticks
dfbreakthr = df[(df['Flash/Onsight'] == True) | (df['Worked Clean'] == True)]
breakthrough_indexes = dfbreakthr.groupby('Rating')['Date'].idxmin().values
df.loc[breakthrough_indexes, "Grade Breakthrough"] = True
df.loc[breakthrough_indexes]

# Count number of attempts to send
# Currently assumes no style lead ticks are fell/hung
# Ignores multipitch
# This counts clean ticks with multiple pitches as total attempts. It also counts fell/hung, and TR with multiple pitches as multiple attempts.
# !!! This will falsely identify a single pitch climb broken into multiple pitches as two attempts, there isn't really a good way to detect this.
df_all_worked_sp_only = df_all_worked[df_all_worked['Listed Pitches'] == 1]
num_to_send = df_all_worked_sp_only.groupby('Route ID')['Pitches'].sum()
matched_attempts = df[df['Worked Clean'] == True].merge(num_to_send, on="Route ID", how="left")
matched_attempts.index = df[df['Worked Clean'] == True].index # I'm dumb and this is the best way I could find to get my index to remain
df.loc[matched_attempts.index, "Attempts"] = matched_attempts.iloc[:,-1]


In [8]:
df.loc[df['Lead Style'].isin(CLEAN_SEND_FIRST), "Attempts"] = ''
df_clean_sends = df[df['Lead Style'].isin(CLEAN_SEND)]
df_clean_sends = df_clean_sends.loc[df_clean_sends.groupby('Route ID')['Date'].idxmin()] # Use only earliest clean send, which will have relevant attempt based data
df_clean_sends['Route Formatted'] = df_clean_sends['Route'].apply(lambda x: string.capwords(x))
df_clean_sends['Location Formatted'] = df_clean_sends['Location'].apply(lambda x: " > ".join(x.title().split(' > ')[-2::]))
df_clean_sends['Date Formatted'] = df_clean_sends['Date'].dt.date

fig = px.bar(df_clean_sends, y="Rating", orientation='h', category_orders={"Rating": YDS_GRADES_LETTER[::-1]}, text='Attempts', custom_data=['Route Formatted', 'Date Formatted', 'Location Formatted', 'Length', 'Avg Stars']) # The [::-1] is an inverse slice
fig.update_layout(font={'family':'Courier New', 'color':'black', 'size':20}, title={'text':'<b>Climbing Pyramid</b>', 'x':0.5, 'font_size':30}, xaxis={'title': 'Number of Routes Sent'}, yaxis={'title': 'Grade'}, paper_bgcolor='#ece5dc', plot_bgcolor='#F5D3A5', bargap=0)
fig.update_traces(marker_color='#7A4F25', textposition = "inside",  hovertemplate='Name: %{customdata[0]}<br>Date: %{customdata[1]}<br>Location: %{customdata[2]}<br>Length: %{customdata[3]}ft<br>Avg Stars: %{customdata[4]}')
# fig.update_traces(marker_color=list(map(lambda x: '#7A4F25' if (x=='') else '#bf9315', df_clean_sends['Attempts'])), textposition = "inside",  hovertemplate='Name: %{customdata[0]}<br>Date: %{customdata[1]}<br>Location: %{customdata[2]}<br>Length: %{customdata[3]}ft<br>Avg Stars: %{customdata[4]}')
fig

In [9]:
fig = px.scatter(df_clean_sends, "Date", "Rating", category_orders={"Rating": YDS_GRADES_LETTER[::-1]}, custom_data=['Route Formatted', 'Date Formatted', 'Location Formatted', 'Length', 'Avg Stars'])
fig.update_layout(font={'family':'Courier New', 'color':'black', 'size':20}, title={'text':'<b>Send by Date</b>', 'x':0.5, 'font_size':30}, xaxis={'title': 'Date'}, yaxis={'title': 'Grade'}, paper_bgcolor='#ece5dc', plot_bgcolor='#F5D3A5', bargap=0)
fig.update_traces(marker_symbol='square', marker_color='#7A4F25', marker_size=12, marker_line_width=2, marker_line_color='black', hovertemplate='Name: %{customdata[0]}<br>Date: %{customdata[1]}<br>Location: %{customdata[2]}<br>Length: %{customdata[3]}ft<br>Avg Stars: %{customdata[4]}')

In [ ]:
# Analyze TR/Lead and OS/FellHung Ratio

# of people that lead, how many onsight/flash? Of people who eventually redpoint, how many tries did it take?

def get_tick_counts(res):
    soup = BeautifulSoup(res.text, 'html.parser')
    try:
        blocks = list(soup.select("#route-stats > div:nth-child(2) > div:nth-last-child(1)")[0].find_all('div'))
    except:
        blocks=[]
    match = [re.search(r'·\s[a-zA-z\/\s]*\.', str(x)) for x in blocks]
    search = [m.group(0)if m else '' for m in match] # pulls match text if match object is not none
    search = [i.lower() for i in search if i] # removes all empty entries, force to lower case for matching strings
    search = [i.split(' / ') for i in search]
    search = [item for sublist in search for item in sublist] # Flatten
    search = [re.sub(r'[^A-Za-z/]', '', substr) for substr in search] # replace all non characters with blank
    search = [x for x in search if x in TICK_OPTIONS] # remove all other falsely identified text entries, old ticks did not require a tick type so it identifies the first sentence instead.
    # I want a list of counts that default to 0 if the tick type is not found, I had to do some crazy transferring to series to dataframes and back to series to make it work.
    search_counts_init = pd.Series(np.nan, index=TICK_OPTIONS).to_frame()
    search_counts = pd.Series(search).value_counts().dropna().to_frame()
    search_counts = search_counts_init.fillna(search_counts).fillna(0).squeeze()
    lead_ratio = search_counts['lead']/(search_counts['follow'] + search_counts['tr'] + search_counts['lead'])
    os_ratio = (search_counts['onsight'] + search_counts['flash']) / (search_counts['onsight'] + search_counts['flash'] + search_counts['fell/hung'] + search_counts['redpoint'] + search_counts['pinkpoint'])
    return pd.Series([lead_ratio, os_ratio])

df[['Lead Ratio', 'OS Ratio']] = df['re statpage'].apply(get_tick_counts)

In [715]:
# Analyze TR/Lead and OS/FellHung Ratio

# of people that lead, how many onsight/flash? Of people who eventually redpoint, how many tries did it take?

def get_tick_details(res):
    name = []
    namelink = []
    entrydate = []
    pitches = []
    style = []
    lead_style = []
    comment = []
    soup = BeautifulSoup(res.text, 'html.parser')
    
    print(soup.select("#route-stats > div.row.pt-main-content > div > h1")) # Tells you which page is being scraped, useful for debugging
    try:
        blocks = list(soup.select("#route-stats > div:nth-child(2) > div:nth-last-child(1)")[0].find_all('tr'))
    except:
        blocks = []
    for x in blocks:
        soup = BeautifulSoup(str(x), 'html.parser')
        entries = soup.find_all('div', attrs={'class': None})
        for entry in entries:
            entrytext = entry.text
            try:
                name.append(soup.find('a').text)
            except:
                name.append('')
                
            try:
                namelink.append(soup.find('a')['href'])
            except:
                namelink.append('')
            
            try:
                date_search = [re.search(r'\w{3}\s\d{1,2},\s\d{4}', entrytext)]
                entrydate.append([subresult.group(0) if subresult else '' for subresult in date_search][0]) # pulls match text if match object is not none
            except:
                entrydate.append('')
            
            try:
                pitches_search = [re.search(r'·([^.]+\s(pitches))', entrytext)] # regex for starting at · and ending at first period only if it includes the word "pitches"
                pitchesinterm = [subresult.group(0) if subresult else '' for subresult in pitches_search]
                pitches.append([int(re.search(r'\d+', subresult).group(0)) if subresult else 1 for subresult in pitchesinterm][0]) # take just the digit of the string
            except:
                pitches.append(1)
            
            try:
                style_search = [re.search(r"·\s\s(solo|tr|follow|lead)", entrytext.lower())]
                style_val = [subresult.group(0)[2:] if subresult else '' for subresult in style_search][0] # I have a conditional in the comment search that depends on this so I made it a separate variable
                style.append(style_val)
            except:
                style.append('')
            
            try:
                if style_val != '':
                    lead_style_search = [re.search(r"/([^.]+)", entrytext.lower())]
                    lead_style.append([subresult.group(0)[2:] if subresult else '' for subresult in lead_style_search][0])
                else:
                    lead_style.append('')
            except:
                lead_style.append('')
            
            try:
                if style_val != '': # If no style comment then entire phrase is the comment.
                    comment_search = [re.search(r"(Solo|TR|Follow|Lead).*", entrytext)]
                    commentinterm = ([subresult.group(0) if subresult else '' for subresult in comment_search])
                    comment.append([re.search(r"\..*", subresult).group(0)[2:] if subresult else '' for subresult in commentinterm][0])
                else:
                    comment_search = [re.search(r"·(.*)", entrytext)]
                    comment.append([subresult.group(0)[2:] if subresult else '' for subresult in comment_search][0])
            except:
                comment.append('')
    # print (len(name),len(namelink),len(entrydate),len(pitches),len(style),len(lead_style),len(comment))
    # print (name,namelink,entrydate,pitches,style,lead_style,comment)
    d = pd.DataFrame({'Username' : name, "User Link" : namelink, "Entry Date": entrydate, "Pitches": pitches, "Style": style, "Lead Style": lead_style, "Comment": comment})
    # One last possible error correction
    d.loc[~d['Lead Style'].isin(TICK_OPTIONS), 'Lead Style'] = ''
    return d

d=get_tick_details(df['re statpage'][22])
d

# df['Statpage df'] = df['re statpage'].apply(get_tick_details)
# df['Statpage df']


[<h1>
            Statistics for 
        Brother From Another Planet
        </h1>]


,Username,User Link,Entry Date,Pitches,Style,Lead Style,Comment
0,Nathan Litster,https://www.mountainproject.com/user/200380857...,"Nov 8, 2022",1,tr,,"So cool, could not pull the roof and could bar..."
1,Gus Meyers,https://www.mountainproject.com/user/200144496...,"Nov 4, 2022",1,tr,,very hard but now you have the sequence
2,Nick G,https://www.mountainproject.com/user/200485331...,"Oct 31, 2022",1,tr,,
3,,,"Oct 21, 2022",1,,,
4,,,"Oct 21, 2022",1,,,
5,,,"Nov 24, 2019",1,,,
6,,,"Apr 2, 2015",1,,,
7,,,"Mar 21, 2015",1,,,
8,Jonathan Wagenet,https://www.mountainproject.com/user/200220022...,"May 9, 2022",1,tr,,
9,Jeff Dunbar,https://www.mountainproject.com/user/105967115...,"Nov 20, 2021",1,lead,fell/hung,"First time on this went great, right up until ..."


In [686]:
extext = blocks[0][2]

name = []
namelink = []
entrydate = []
pitches = []
style = []
lead_style = []
comment = []

soup = BeautifulSoup(str(extext), 'html.parser')
entries = soup.find_all('div', attrs={'class': None})

for entry in entries:
    entrytext = entry.text
    
    namelink.append(soup.find('a')['href'])
    name.append(soup.find('a').text)
    
    date_search = [re.search(r'\w{3}\s\d{1,2},\s\d{4}', entrytext)]
    entrydate.append([subresult.group(0)if subresult else '' for subresult in date_search][0]) # pulls match text if match object is not none
    
    pitches_search = [re.search(r'·([^.]+\s(pitches))', entrytext)] # regex for starting at · and ending at first period only if it includes the word "pitches"
    pitchesinterm = [subresult.group(0) if subresult else '' for subresult in pitches_search]
    pitches.append([int(re.search(r'\d+', subresult).group(0)) if subresult else 1 for subresult in pitchesinterm][0]) # take just the digit of the string
    
    style_search = [re.search(r"(solo|tr|follow|lead)", entrytext.lower())]
    style.append([subresult.group(0) if subresult else '' for subresult in style_search][0])
    
    lead_style_search = [re.search(r"/([^.]+)", entrytext.lower())]
    lead_style.append([subresult.group(0)[2:] if subresult else '' for subresult in lead_style_search][0])
    
    comment_search = [re.search(r"(solo|tr|follow|lead).*", entrytext.lower())]
    commentinterm = ([subresult.group(0)[2:] if subresult else '' for subresult in comment_search])
    comment.append([re.search(r"\..*", subresult).group(0)[2:] if subresult else '' for subresult in commentinterm][0])
    
    
# print (len(name),len(namelink),len(entrydate),len(pitches),len(style),len(lead_style),len(comment))
# print (name,namelink,entrydate,pitches,style,lead_style,comment)
d = pd.DataFrame({'Username' : name, "User Link" : namelink, "Entry Date": entrydate, "Pitches": pitches, "Style": style, "Lead Style": lead_style, "Comment": comment})